In [1]:
import numpy as np
import simpy
import random

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [23]:
SEED = 930
UNIFORM_INTERVAL = [4.17, 5.37]
NUM_SERVER = 2
SERVER_UTILIZATIONS_PHASE1 = [0.6, 0.7, 0.8, 0.9]
SERVER_SERVICE_MEAN_PHASE1 = [5.725,6.679,7.634,8.588]
SERVER_UTILIATION_PHASE2 = 0.8
NUMBER_OF_CUSTOMERS = 100


In [24]:
phase_1_lib = {"num_server":NUM_SERVER,"arr_interval":UNIFORM_INTERVAL,"util":SERVER_UTILIZATIONS_PHASE1,"serv_mean":SERVER_SERVICE_MEAN_PHASE1}
phase_2_lib = {"num_server":NUM_SERVER,"arr_interval":UNIFORM_INTERVAL,"util":SERVER_UTILIATION_PHASE2,"serv_mean":7.634}

In [34]:
class Job(object):
    def __init__(self, name, env, operator , run_):
        self.env = env
        self.name = name
        self.operator = operator
        self.arrival_t = self.env.now
        self.action = self.env.process(self.call())
        self.run = run_
        
    
    def call(self):
        print('%s initiated at %g' % (self.name, self.env.now))
 
        with self.operator.request() as req:
            yield req
            print('%s is assigned to an operator at %g' % (self.name, self.env.now))
            self.run.queue_w_times.append(self.env.now - self.arrival_t)
            yield self.env.process(self.progress_job())
            print('%s is done at %g' % (self.name, self.env.now))
            
    def progress_job(self):
        duration = random.expovariate(self.run.serv_rate)
        yield self.env.timeout(duration)
        self.run.service_times.append(duration)
        self.departure_t = self.env.now
        
        

In [35]:
class Run():
    def __init__(self,arr_rate,serv_rate):
        self.seed = random.random()*425
        self.run = False
        self.arr_rate = arr_rate
        self.serv_rate = 1/serv_rate
        self.jobs=[]
        self.service_times=[]
        self.queue_w_times=[]
        self.env = simpy.Environment()
        self.operator = simpy.Resource(self.env, capacity = NUM_SERVER)
    
    def run_sim(self):
        random.seed(self.seed)
        self.env.process(self.jobs_generator())
        self.env.run()
        self.run = True
        
    def show_run(self):
        if self.run!=True:
            self.run_sim()
        
        
    def jobs_generator(self):
        for i in range(NUMBER_OF_CUSTOMERS):
            yield self.env.timeout(random.expovariate(self.arr_rate))
            job = Job('Job %s' %(i+1), self.env, self.operator,self)
            self.jobs.append(job)  

In [84]:
class Phase():
    def __init__(self,lib,num_runs):
        self.num_server = lib["num_server"]
        self.arr_int = 1/((lib["arr_interval"][0]+lib["arr_interval"][1])/2)
        self.util = lib["util"]
        self.serv_mean = lib["serv_mean"]
        self.init_runs(num_runs)
        self.num_runs = num_runs

    def init_runs(self,num_runs):
        self.runs = []
        for util_no in range(len(self.util)):
            for i in range(num_runs):
                obj = Run(self.arr_int,self.serv_mean[util_no])
                self.runs.append(obj)

    def run_phase(self):
        for run in self.runs:
            run.run_sim()
        print("total run :",len(self.runs)," ")
    
    def ensemble_averages(self):
        self.ens_avg = [[0 for i in range(NUMBER_OF_CUSTOMERS)] for j in range(len(self.util))]
        for k in range(len(self.util)):
            for run_no in range(self.num_runs):
                for i in range(NUMBER_OF_CUSTOMERS):
                    self.ens_avg[k][i] = self.ens_avg[k][i] + (self.runs[k*run_no].jobs[i].departure_t - self.runs[k*run_no].jobs[i].arrival_t)
        self.ens_avg = np.divide(self.ens_avg,self.num_runs)
            

In [85]:
phase_1 = Phase(phase_1_lib,10)

In [86]:
phase_1.run_phase()

Job 1 initiated at 2.8607
Job 1 is assigned to an operator at 2.8607
Job 1 is done at 4.71938
Job 2 initiated at 9.5573
Job 2 is assigned to an operator at 9.5573
Job 3 initiated at 10.1601
Job 3 is assigned to an operator at 10.1601
Job 2 is done at 12.1151
Job 4 initiated at 14.6711
Job 4 is assigned to an operator at 14.6711
Job 4 is done at 15.0334
Job 5 initiated at 16.5206
Job 5 is assigned to an operator at 16.5206
Job 6 initiated at 18.4955
Job 7 initiated at 19.7745
Job 5 is done at 20.2555
Job 6 is assigned to an operator at 20.2555
Job 8 initiated at 20.3683
Job 6 is done at 24.2004
Job 7 is assigned to an operator at 24.2004
Job 7 is done at 27.2013
Job 8 is assigned to an operator at 27.2013
Job 8 is done at 27.2399
Job 9 initiated at 37.0829
Job 9 is assigned to an operator at 37.0829
Job 9 is done at 37.8459
Job 10 initiated at 44.895
Job 10 is assigned to an operator at 44.895
Job 10 is done at 45.5921
Job 11 initiated at 46.5881
Job 11 is assigned to an operator at 46.

In [87]:
phase_1.ensemble_averages()
print(len(phase_1.ens_avg),len(phase_1.ens_avg[0]))
np.mean(phase_1.ens_avg,axis=1)

4 100


array([14.9909946 ,  8.89453805, 10.37120265, 11.12153495])

AttributeError: 'list' object has no attribute 'service_times'